# Лекция 11: Ансамбли деревьев. Случайный Лес

**Цель лекции:**
1.  Познакомиться с концепцией **ансамблевых методов** как способом повышения качества и стабильности моделей.
2.  Разобрать метод **бэггинга (Bagging)** как основу для случайного леса.
3.  Изучить алгоритм **Случайный Лес (Random Forest)**, его преимущества и практическое применение.
4.  Кратко коснуться альтернативного подхода — **бустинга (Boosting)**.

### Часть 1: Почему одного дерева недостаточно? Мудрость толпы

На прошлой лекции мы выяснили, что у деревьев решений есть серьезный недостаток: они **нестабильны** и сильно склонны к **переобучению**. Небольшое изменение в данных может кардинально изменить структуру дерева.

**Как решить эту проблему?** Давайте обратимся к житейской мудрости. Когда мы хотим принять важное решение, мы редко полагаемся на мнение одного эксперта. Мы собираем мнения нескольких, и итоговое решение принимаем на основе консенсуса. Этот принцип называется **"мудрость толпы"**.

В машинном обучении этот подход называется **ансамблированием**. Идея состоит в том, чтобы обучить множество разных (слабых или нестабильных) моделей и объединить их предсказания, чтобы получить одно, но более точное и робастное.

![ensamble](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/ensamble.png)

### Часть 2: Бэггинг (Bagging = Bootstrap Aggregating)

Бэггинг — один из самых популярных ансамблевых методов и основа для случайного леса. Его название состоит из двух частей.

#### Шаг 1: Bootstrap (Бутстрэп)

Чтобы наши модели (деревья) получились разными, нам нужно обучить их на **разных данных**. Но у нас есть только одна обучающая выборка. Где взять другие?

Бутстрэп — это статистический метод, который позволяет из одной выборки размером `N` сгенерировать множество **подвыборок** того же размера `N`. Это делается путем **случайного выбора объектов из исходной выборки с возвращением**.

![bootstrap](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/bootstrap1.png)
Еще пример
![bootstrap1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/bootstrapping.png)

В результате каждая подвыборка будет уникальной: некоторые исходные объекты в ней будут повторяться, а некоторые (в среднем около 37%) не попадут вовсе.

#### Шаг 2: Aggregation (Агрегирование)

На каждой из `T` полученных бутстрэп-подвыборок мы обучаем **свое независимое дерево решений** $b_t(x)$. Обычно эти деревья делают очень глубокими, намеренно позволяя им переобучиться на своей части данных.

Когда нам нужно сделать предсказание для нового объекта, мы собираем все `T` предсказаний и агрегируем их.

**Для классификации — мажоритарное голосование:**
Итоговым предсказанием $a(x)$ становится тот класс, за который проголосовало большинство деревьев.
$$ a(x) = \text{sign} \left( \frac{1}{T} \sum_{t=1}^{T} b_t(x) \right) $$

**Для регрессии — усреднение:**
Итоговым предсказанием $a(x)$ становится среднее арифметическое предсказаний всех деревьев.
$$ a(x) = \frac{1}{T} \sum_{t=1}^{T} b_t(x) $$

В результате ошибки и "странности" отдельных переобученных деревьев взаимно компенсируют друг друга, и итоговое предсказание ансамбля становится гораздо более точным и стабильным.

#### Математическое обоснование: как усреднение уменьшает ошибку

Почему "мудрость толпы" работает? Этот эффект можно легко объяснить с помощью базовой теории вероятностей.

Предположим, мы решаем задачу регрессии. Пусть:
*   **$\theta$** — это истинное, правильное значение, которое мы хотим предсказать.
*   **$x_j$** — это ответ $j$-го "эксперта" (или $j$-го дерева в нашем ансамбле).

Каждый ответ можно представить в виде аддитивной модели:
$$ x_j = \theta + \eta_j $$
где $\eta_j$ — это случайная ошибка $j$-го эксперта.

Для простоты сделаем два разумных допущения:
1.  **Ошибки несмещенные:** В среднем, эксперты не ошибаются систематически в одну и ту же сторону. Математически: среднее значение ошибки равно нулю, $E[\eta_j] = 0$.
2.  **Ошибки независимы:** Ошибка одного эксперта не связана с ошибкой другого. Математически: $E[\eta_i \eta_j] = 0$ для $i \neq j$. Это ключевое условие, для выполнения которого мы и используем бутстрэп и `max_features`!

Пусть **дисперсия** ошибки каждого отдельного эксперта равна $\sigma^2$. Дисперсия — это мера "разброса" или нестабильности предсказаний.
$$ \sigma^2 = E[\eta_j^2] $$

Теперь найдем предсказание нашего ансамбля, усреднив ответы $T$ экспертов:
$$ y = \frac{1}{T} \sum_{j=1}^{T} x_j $$

Какова будет дисперсия ошибки этого усредненного ответа?
$$ E[(\theta - y)^2] = E\left[\left(\theta - \frac{1}{T} \sum_{j=1}^{T} x_j\right)^2\right] = E\left[\left(\frac{1}{T} \sum_{j=1}^{T} (\theta - x_j)\right)^2\right] = E\left[\left(\frac{1}{T} \sum_{j=1}^{T} \eta_j\right)^2\right] $$

Раскрывая квадрат суммы и используя свойство независимости ошибок, мы получаем:
$$ \frac{1}{T^2} \sum_{j=1}^{T} E[\eta_j^2] = \frac{1}{T^2} \cdot T \cdot \sigma^2 = \frac{\sigma^2}{T} $$

**Итог:**
> Дисперсия усредненного ответа (ансамбля) в **$T$ раз меньше**, чем дисперсия ответа одного отдельного дерева.

**Простыми словами:** Усредняя предсказания $T$ независимых, но шумных моделей, мы уменьшаем итоговый "шум" (ошибку) в $T$ раз. Именно поэтому бэггинг и случайный лес так эффективно борются с нестабильностью и переобучением одиночных деревьев. Чем больше деревьев мы добавляем, тем сильнее сглаживаются их индивидуальные ошибки.

### Часть 3: Случайный Лес (Random Forest) — Бэггинг с "изюминкой"

**Случайный лес** — это усовершенствованная версия бэггинга деревьев. Он добавляет еще один элемент случайности, чтобы сделать деревья еще более независимыми друг от друга.

**Отличие от бэггинга:**
При построении **каждого узла** в каждом дереве, алгоритм выбирает лучший сплит не из **всех** доступных признаков, а только из **случайного подмножества** этих признаков.

Этот параметр в `scikit-learn` называется `max_features`.

**Зачем это нужно?**
Представьте, что в ваших данных есть один очень сильный, доминирующий признак. В обычном бэггинге большинство деревьев выберет именно его для первого разбиения в корне. В результате все деревья будут очень похожи друг на друга ("скоррелированы"), и эффект от усреднения будет слабее.

Случайный лес заставляет некоторые деревья строиться **без доступа** к этому сильному признаку, вынуждая их находить другие, альтернативные закономерности в данных. Это увеличивает "разнообразие мнений" в ансамбле и, как правило, повышает итоговую точность.

![randomforest1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/random_forest1.png)
![randomforest1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/randomforest1.png)
![randomforest1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/randomforest2.png)

### Часть 4: Историческая справка и реализация в Scikit-Learn

#### Ключевые фигуры и идеи
*   **CART (Classification and Regression Trees):** Алгоритм, лежащий в основе построения отдельных деревьев в `scikit-learn`, был предложен **Лео Брейманом** и др. в 1984 году.
*   **Random Subspace Method:** Идея обучать классификаторы на случайных подмножествах признаков была предложена **Тин Кам Хо** в 1998 году. Это и есть ядро `max_features`.
*   **Бэггинг и Случайный Лес:** **Лео Брейман** в 1994 году предложил бэггинг, а в 2001 году объединил его с методом случайных подпространств, формализовав и популяризовав алгоритм **Random Forest**.

#### Реализация в Scikit-Learn
В `scikit-learn` есть два основных класса:
- `sklearn.ensemble.RandomForestClassifier` — для задач классификации.
- `sklearn.ensemble.RandomForestRegressor` — для задач регрессии.

**Ключевые гиперпараметры:**
- `n_estimators`: Количество деревьев в лесу. Самый важный параметр. Чем больше, тем лучше (до определенного предела), но тем дольше обучение.
- `max_features`: Количество признаков, которые рассматриваются при поиске лучшего сплита. Типичные значения: `'sqrt'` (квадратный корень из общего числа признаков), `'log2'`, или конкретное число.
- `criterion`: Критерий для оценки качества сплита (`'gini'` или `'entropy'` для классификации, `'squared_error'` для регрессии).
- `max_depth`, `min_samples_split`, `min_samples_leaf`: Параметры для регуляризации каждого отдельного дерева (аналогично `DecisionTreeClassifier`).
- `bootstrap`: `True` (по умолчанию) или `False`. Включает или выключает механизм бутстрэпа.
- `oob_score`: `True` или `False`. Включает или выключает оценку Out-of-Bag.

#### Что такое Out-of-Bag (OOB) Score?

Это одна из самых элегантных особенностей Случайного Леса. 

**Идея:** При создании бутстрэп-подвыборки для обучения дерева №1, около 37% исходных данных в нее не попали. Эти "отложенные" данные можно использовать как **валидационный набор** специально для этого дерева №1. 

**Процесс:**
1.  Для **каждого** объекта $x_i$ из исходной обучающей выборки мы находим все те деревья, при обучении которых этот объект **не использовался** (был "out-of-bag").
2.  Мы просим эти деревья сделать предсказание для объекта $x_i$.
3.  Усредняем их предсказания, чтобы получить итоговое "OOB-предсказание" для объекта $x_i$.
4.  Сравниваем все OOB-предсказания с реальными метками $y_i$ и получаем **Out-of-Bag Score** — честную оценку качества модели, полученную без необходимости создавать отдельный валидационный набор!

Чтобы использовать эту возможность, нужно установить `bootstrap=True` (что является значением по умолчанию) и `oob_score=True`.

![ensamble](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/randomforest4.png)

#### Глубокое погружение: Зачем нужен `max_features` и откуда взялись значения?

Параметр `max_features` — это та самая "изюминка", которая отличает Случайный Лес от простого Бэггинга деревьев. Давайте разберемся, зачем он нужен и почему для него используются такие странные значения, как $\sqrt{n_{features}}$.

**Главная цель: Декорреляция деревьев**

Как мы помним, "мудрость толпы" работает лучше всего, когда мнения в этой толпе **независимы** и **разнообразны**. Если все "эксперты" (деревья) будут думать одинаково, то и ансамбль не будет умнее, чем один-единственный эксперт.

**Проблема:** Представьте, что в ваших данных есть один очень сильный, доминирующий признак (например, "доход" при предсказании покупки дорогого товара). Если мы будем строить деревья методом простого бэггинга, то почти каждое дерево, скорее всего, выберет именно этот признак для самого первого, корневого разбиения. В результате все деревья в лесу будут очень похожи друг на друга — они станут **скоррелированными**. Эффективность ансамбля от этого сильно упадет.

**Решение:** Параметр `max_features` решает эту проблему, искусственно ограничивая "выбор" для каждого узла. При построении очередного узла в очередном дереве алгоритм делает следующее:
1.  Берет **все** признаки, доступные в датасете.
2.  Выбирает из них **случайное подмножество** размером `max_features`.
3.  Ищет лучший сплит **только среди признаков из этого подмножества**.

Это заставляет некоторые деревья строиться, даже не "видя" доминирующего признака, и вынуждает их находить другие, менее очевидные закономерности в данных. Это и есть **декорреляция**.

**Основной компромисс (Trade-off)**

Выбор `max_features` — это баланс между силой отдельных деревьев и их разнообразием:

*   **Маленькое значение `max_features`:**
    *   **Плюс:** Деревья получаются очень разными (низкая корреляция). Это хорошо для "мудрости толпы".
    *   **Минус:** Отдельные деревья могут получиться очень "слабыми", так как у них может не быть доступа к важным признакам. Их индивидуальная предсказательная сила падает.

*   **Большое значение `max_features`:**
    *   **Плюс:** Каждое дерево получается сильным, так как имеет доступ почти ко всем признакам.
    *   **Минус:** Деревья становятся очень похожими друг на друга (высокая корреляция), и ансамбль теряет свою эффективность. Если `max_features` равно общему числу признаков, то мы получаем обычный Бэггинг.

**Откуда взялись значения $\sqrt{n}$ и $\log_2(n)$?**

Эти значения — не результат строгой математической теоремы, а **эвристики** (практические правила), которые были предложены **Лео Брейманом** в его оригинальной работе по Случайному Лесу. Они были найдены эмпирически, в ходе множества экспериментов на разных наборах данных, и зарекомендовали себя как отличная отправная точка.

*   `max_features='sqrt'` (или $\sqrt{n_{features}}$):
    *   **Почему это хорошо?** Это хороший компромисс. Количество выбираемых признаков растет с общим числом признаков, но растет медленно. Это позволяет сохранить разнообразие деревьев, но при этом дает им достаточно хороший шанс "увидеть" важные предикторы.
    *   **Когда используется?** Это значение по умолчанию в `scikit-learn` для **задач классификации**, и в большинстве случаев его не нужно менять.

*   `max_features='log2'` (или $\log_2(n_{features})$):
    *   **Почему это хорошо?** Это еще более жесткое ограничение. Количество выбираемых признаков растет очень медленно.
    *   **Когда используется?** Этот вариант может быть полезен, когда у вас **очень много признаков** (тысячи), и вы подозреваете, что многие из них либо неинформативны (шум), либо сильно скоррелированы между собой.

*   `max_features=1.0` (или `n_features`):
    *   **Что это?** Это эквивалент обычного Бэггинга. Алгоритм на каждом шаге видит все признаки.
    *   **Когда используется?** Используется по умолчанию в `scikit-learn` для **задач регрессии**. Считается, что для регрессии важнее иметь более сильные индивидуальные деревья, и проблема корреляции стоит не так остро.

### Часть 6: Кратко о Бустинге (Boosting)

Бустинг — это другой мощный подход к ансамблированию, который кардинально отличается от бэггинга.

| Бэггинг (Random Forest) | Бустинг (Gradient Boosting, AdaBoost) |
| :--- | :--- |
| Деревья строятся **независимо** и **параллельно**. | Деревья строятся **последовательно**. |
| Каждое дерево \"не знает\" о других. | Каждое следующее дерево **учится на ошибках** предыдущего. |
| Цель — уменьшить **дисперсию (variance)** за счет усреднения. | Цель — уменьшить **смещение (bias)** за счет исправления ошибок. |
| Используются глубокие, переобученные деревья. | Используются очень простые, \"слабые\" деревья (часто \"пни\" с глубиной 1-3). |

#### Интуиция бустинга: Работа над ошибками

**Интуиция:** Представьте, что вы собираете команду экспертов для решения сложной задачи. Вместо того чтобы просить их работать параллельно, вы делаете это последовательно:
1.  Первый эксперт делает свою попытку.
2.  Вы смотрите на его ошибки и просите второго эксперта сосредоточиться именно на тех случаях, где первый потерпел неудачу.
3.  Третий эксперт концентрируется на ошибках, которые остались после работы первых двух.
4.  И так далее. Итоговое решение — это взвешенное "мнение" всех экспертов.

#### Пример: Алгоритм AdaBoost (Adaptive Boosting)

AdaBoost — один из первых и самых наглядных алгоритмов бустинга.

1.  **Инициализация:** В самом начале все объекты обучающей выборки имеют одинаковый **вес** $w_i = 1/N$.

2.  **Итеративное построение:** Для каждого шага $t=1, \dots, T$:
    a. Обучается "слабая" модель (например, дерево-пень) $b_t(x)$ на данных с учетом текущих весов $w_i$. Модель будет стараться правильно классифицировать объекты с большими весами.
    б. Вычисляется ошибка $\epsilon_t$ этой модели.
    в. Вычисляется **"вес голоса"** этой модели $\alpha_t$. Чем меньше ошибка модели, тем "громче" будет ее голос в финальном ансамбле.
       $$ \alpha_t = \frac{1}{2} \ln{\frac{1 - \epsilon_t}{\epsilon_t}} $$
    г. **Обновляются веса объектов:**
       *   Веса **неправильно** классифицированных объектов **увеличиваются**.
       *   Веса **правильно** классифицированных объектов **уменьшаются**.
       Это заставляет следующую модель $b_{t+1}(x)$ сконцентрироваться на самых "сложных" объектах.

3.  **Финальное предсказание:** Итоговая модель — это взвешенное голосование всех слабых моделей.

    $$ a(x) = \text{sign} \left( \sum_{t=1}^{T} \alpha_{t} b_{t}(x) \right) $$

**Gradient Boosting** развивает эту идею: каждая следующая модель учится предсказывать не просто ошибки, а **градиент функции потерь** предыдущей модели. Это более общий и, как правило, более мощный подход.

Это очень мощные, но более сложные для настройки и понимания методы. Популярные реализации — Gradient Boosting, XGBoost, LightGBM. В рамках нашего курса мы их подробно рассматривать не будем, но это важное направление для дальнейшего изучения.

![baggingboosting](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/baggingboosting.png)

### Часть 4.1: "Бесплатный обед" Случайного Леса: почему он не переобучается с ростом числа деревьев?

Вспомним главную проблему одного дерева решений: чем глубже (сложнее) дерево, тем выше риск переобучения. Интуитивно кажется, что добавляя в ансамбль все больше и больше сложных, переобученных деревьев, мы должны сделать итоговую модель еще более переобученной.

Но со Случайным Лесом происходит удивительная вещь: **с увеличением количества деревьев (`n_estimators`) ошибка на тестовой выборке (обобщающая способность) перестает расти и выходит на плато.**

**Почему так происходит?**

Этот эффект является следствием **Закона больших чисел**. Каждое дерево в лесу — это, по сути, случайная величина, которая делает свое предсказание. Оно имеет высокое смещение (variance), то есть сильно подогнано под свою бутстрэп-выборку.

Когда мы усредняем предсказания большого количества **независимых (или слабо скоррелированных)** случайных величин, их индивидуальные ошибки (дисперсия) начинают взаимно уничтожать друг друга.

*   **Первые деревья (от 1 до ~20):** Каждое новое дерево приносит много новой, уникальной "информации". Ошибка на тестовой выборке резко падает.
*   **Следующие деревья (~20 до ~100-200):** Новые деревья все еще вносят свой вклад, улучшая результат, но эффект уже не такой сильный. Ошибка продолжает плавно снижаться.
*   **Дальнейшее увеличение числа деревьев:** Ансамбль становится достаточно "мудрым". Каждое новое дерево уже не добавляет принципиально новой информации, а лишь незначительно "уточняет" общую картину. Ошибка на тестовой выборке стабилизируется и выходит на **асимптотическое плато**. Она больше не ухудшается!

**Вывод:** В отличие от многих других моделей, где нужно искать компромисс со сложностью, для `n_estimators` в Случайном Лесе правило простое: **"чем больше, тем лучше"** (или, по крайней мере, не хуже). Единственным ограничением становится время обучения.

Давайте посмотрим на это на практике.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Генерируем более сложный датасет
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=5,
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Диапазон количества деревьев для проверки
n_estimators_range = range(1, 201)

# Списки для хранения ошибок
train_errors = []
test_errors = []

for n_estimators in n_estimators_range:
    # Обучаем модель с текущим количеством деревьев
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Делаем предсказания
    train_preds = rf.predict(X_train)
    test_preds = rf.predict(X_test)
    
    # Считаем ошибку (1 - accuracy) и добавляем в списки
    train_errors.append(1 - accuracy_score(y_train, train_preds))
    test_errors.append(1 - accuracy_score(y_test, test_preds))

# Рисуем график
plt.figure(figsize=(12, 7))
plt.plot(n_estimators_range, train_errors, label="Ошибка на обучающей выборке (Train Error)")
plt.plot(n_estimators_range, test_errors, label="Ошибка на тестовой выборке (Test Error)", linewidth=2)

plt.ylabel("Уровень ошибки (1 - Accuracy)")
plt.xlabel("Количество деревьев (n_estimators)")
plt.title("Зависимость ошибки от количества деревьев в Случайном Лесу", fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

### Часть 5: Случайный Лес в действии (практический пример)

Давайте сравним разделяющую поверхность одного дерева и случайного леса на одних и тех же данных.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# Генерируем данные с высоким уровнем шума, чтобы задача была сложной
X, y = make_moons(n_samples=500, noise=0.3, random_state=42)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# --- ИСПРАВЛЕНИЕ: Обучаем модели ТОЛЬКО на обучающей выборке ---
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_train)
# ----------------------------------------------------------------

# --- Визуализация разделяющих поверхностей ---
def plot_boundary(ax, model, X_data, y_data, title):
    x_min, x_max = X_data[:, 0].min() - 0.5, X_data[:, 0].max() + 0.5
    y_min, y_max = X_data[:, 1].min() - 0.5, X_data[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='winter')
    # Отображаем только тестовые точки, чтобы видеть, как модели их классифицируют
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, s=30, edgecolor='k', cmap='winter')
    ax.set_title(title)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
plot_boundary(axes[0], tree_clf, X, y, "Одно Дерево Решений")
plot_boundary(axes[1], rf_clf, X, y, "Случайный Лес (100 деревьев)")
plt.suptitle("Сравнение разделяющих поверхностей (на тестовых данных)", fontsize=16)
plt.show()


# --- Оценка моделей на тестовой выборке ---
y_pred_tree = tree_clf.predict(X_test)
y_pred_rf = rf_clf.predict(X_test)

print("="*60)
print("Classification Report: Одно Дерево Решений")
print(classification_report(y_test, y_pred_tree))
print("="*60)

print("\n" + "="*60)
print("Classification Report: Случайный Лес")
print(classification_report(y_test, y_pred_rf))
print("="*60)


# --- Матрицы ошибок ---
cm_tree = confusion_matrix(y_test, y_pred_tree)
cm_rf = confusion_matrix(y_test, y_pred_rf)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

disp_tree = ConfusionMatrixDisplay(confusion_matrix=cm_tree)
disp_tree.plot(ax=axes[0], cmap='Blues', values_format='d')
axes[0].set_title("Матрица ошибок - Дерево")

disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf)
disp_rf.plot(ax=axes[1], cmap='Blues', values_format='d')
axes[1].set_title("Матрица ошибок - Случайный лес")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Генерируем данные с высоким уровнем шума
X, y = make_moons(n_samples=500, noise=0.3, random_state=42)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 1. Обучаем "отдельно стоящее" Дерево Решений
# Мы не ограничиваем глубину, чтобы оно переобучилось
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

# 2. Обучаем Случайный Лес
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_train)


# --- ВИЗУАЛИЗАЦИЯ ДЕРЕВЬЕВ ---

# 3. Рисуем "отдельно стоящее" Дерево Решений
# Ограничим глубину отображения (max_depth=3), чтобы график был читаемым,
# но помним, что на самом деле дерево гораздо глубже.
plt.figure(figsize=(20, 10))
plot_tree(tree_clf, 
          max_depth=3, # Ограничение только для визуализации
          feature_names=['feature_0', 'feature_1'], 
          class_names=['Класс 0', 'Класс 1'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Верхние уровни 'отдельно стоящего' Дерева Решений", fontsize=16)
plt.show()


# 4. "Достаем" одно дерево из леса и рисуем его
# rf_clf.estimators_ - это список всех деревьев в лесу. Возьмем первое.
first_tree_from_forest = rf_clf.estimators_[0]

plt.figure(figsize=(20, 10))
plot_tree(first_tree_from_forest, 
          max_depth=3, # Ограничение только для визуализации
          feature_names=['feature_0', 'feature_1'],
          class_names=['Класс 0', 'Класс 1'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Верхние уровни одного дерева ИЗ Случайного Леса", fontsize=16)
plt.show()

**Вывод:** Граница одного дерева — "рваная" и сложная, она пытается идеально подстроиться под каждую точку. Граница Случайного Леса — гораздо более гладкая и обобщенная. Она отражает общую тенденцию в данных, а не шум. Структура одного дерева из случайного леса заметно отличается от структуры дерева, обученного на всех данных, из-за применения бутстрэпа и случайного выбора признаков на каждом узле. Именно это принудительное "разнообразие мнений" между деревьями и позволяет ансамблю достигать высокой точности и устойчивости к переобучению.

### Часть 7: Преимущества и Недостатки Случайного Леса

#### Преимущества:
*   **Высокая точность прогнозов:** На большинстве задач работает лучше линейных алгоритмов; точность сравнима с точностью бустинга.
*   **Устойчивость к выбросам и шуму:** Практически не чувствителен к выбросам в данных из-за случайного сэмплирования выборок методом бутстрэпа.
*   **Нечувствительность к масштабированию** и другим монотонным преобразованиям значений признаков.
*   **Работает «из коробки»:** Не требует тщательной настройки параметров для получения хорошего результата.
*   **Эффективно работает с большим числом признаков и классов.**
*   **Редко переобучается:** На практике добавление деревьев почти всегда только улучшает композицию.
*   **Хорошо работает с пропущенными данными.**
*   **Легко распараллеливать** и масштабировать.

#### Недостатки:
*   **Сложность интерпретации:** В отличие от одного дерева, результаты случайного леса сложнее интерпретировать.
*   **Проблемы с разреженными данными:** Работает хуже многих линейных методов, когда в выборке очень много разреженных признаков (например, в задачах анализа текстов).
*   **Не умеет экстраполировать:** Как и одиночные деревья, лес не может предсказывать значения за пределами диапазона, который он видел в обучающей выборке.
*   **Предвзятость к категориальным признакам:** Склонен отдавать предпочтение категориальным признакам с большим количеством уровней (категорий).
*   **Большой размер модели:** Требует O(N·K) памяти для хранения, где N - число деревьев, а K - число узлов в них.